<h1 align="center"> <strong> XIOS Python Interface Demo </strong> </h1>

This Notebook contains a demonstration of the XIOS Python interface features. 
Note that you must have the iodef.xml associated file in this curent directory to run this notebook

## **1. Necessary imports**

In [1]:
import xios
import numpy as np

Library /lustre/fswork/projects/rech/psl/upu87pm/XIOS3/dev/xios4py/lib/libxios.so successfully loaded.


## **2. XIOS Initialization**

In [2]:
xios.initialize('client')
context = xios.Context('context')

In [5]:
context.calendar.start_date

## **3. Calendar definition**

In [6]:
context.calendar.timestep    = xios.Duration(second = 36000)

In [7]:
context.calendar.timestep

36000s

In [8]:
context.calendar.type = 'D360'

## **4. Attribute's setter and getter**

In [9]:
axis = xios.Axis('axis', comment = 'I am the comment')

In [10]:
axis.id

axis

In [11]:
axis_g = xios.AxisGroup('axis_g')

In [12]:
axis_child = axis_g.add_axis('axis_child')

In [13]:
axis_child.id

'axis_child'

In [14]:
scalar = xios.Scalar('scalar')

In [15]:
print("axis value : ",axis.value)

axis value :  [ 100.  200.  300.  400.  500.  600.  700.  800.  900. 1000.]


In [16]:
domain = xios.Domain('domain')
print("domain size = ", domain.ni_glo, "*", domain.nj_glo)

domain size =  60 * 20


In [17]:
domain.lonvalue_1d = 360*np.arange(domain.ni_glo)/domain.ni_glo - 180
domain.latvalue_1d = 180*np.arange(domain.nj_glo)/domain.nj_glo - 90

## **5. Close context definition**

In [18]:
context.close_definition()

In [19]:
context.calendar.type

D360

In [20]:
context.calendar.comment = 'hey'

In [21]:
context.calendar.comment

hey

In [22]:
context.calendar.timestep = xios.Duration(second = 99)

In [23]:
context.calendar.timestep

99s

## **6. Sending field**

In [13]:
nx, ny, nz = domain.ni_glo, domain.nj_glo, axis.n_glo
x, y, z = np.indices((nx, ny, nz))
def data(t):
    center_shift = 20 * np.sin(2 * np.pi * t / 200)  
    return np.sqrt((x - nx//2 - center_shift)**2 +
                   (y - ny//2)**2 +
                   (z - nz//2)**2)

field = xios.Field('field')

In [14]:
for ts in range(200):
   xios.update_calendar(ts)
   field.send(data(ts))

## **7. Context and XIOS finalization**

In [15]:
context.finalize()
xios.finalize()

Server Context destructor


## **8. Output vizualization** 

In [1]:
import xarray as xr
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

ds = xr.open_dataset("output.nc")
field = ds["field"]

def plot_slice(time_idx=0, axis_idx=0):
    slice_2d = field.isel(time_counter=time_idx, axis=axis_idx)
    slice_2d.plot(cmap="plasma")
    plt.title(f"field at time={time_idx}, axis={axis_idx}")
    plt.show()

widgets.interact(plot_slice,
                 time_idx=(0, field.sizes["time_counter"] - 1),
                 axis_idx=(0, field.sizes["axis"] - 1))



KeyError: "No variable named 'field'. Did you mean one of ('field06_src', 'field06_dst', 'field05_src', 'field05_dst', 'field04_src', 'field04_dst', 'field03_src', 'field03_dst', 'field02_src', 'field02_dst')?"

In [2]:
import xarray as xr
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# Load the dataset
ds = xr.open_dataset("dia_torc_to_tico_OceTemp_1stOrder.nc")

# Collect field variable names
field_names = [name for name in ds.data_vars if name.startswith("field")]

# Create a dropdown for fields
field_dropdown = widgets.Dropdown(
    options=field_names,
    value=field_names[0],
    description='Field:',
)

# Toggle for source/destination
def plot_field(field_name):
    field = ds[field_name]
    
    # Try to auto-detect correct lat/lon for plotting
    if 'y_domain_src' in field.dims and 'x_domain_src' in field.dims:
        lat = ds['nav_lat_domain_src']
        lon = ds['nav_lon_domain_src']
        coords = {'lat': lat, 'lon': lon}
        field = field.assign_coords(coords)
        field.plot(cmap="plasma")
    elif 'cell_domain_dst' in field.dims:
        lat = ds['lat_domain_dst']
        lon = ds['lon_domain_dst']
        field = field.assign_coords({'cell': field['cell_domain_dst'], 'lat': lat, 'lon': lon})
        plt.scatter(lon, lat, c=field, cmap='plasma', s=5)
        plt.colorbar(label=field_name)
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.title(field_name)
    else:
        print("Unrecognized grid layout.")
        return
    
    plt.show()

# Create interactivity
widgets.interact(plot_field, field_name=field_dropdown)


interactive(children=(Dropdown(description='Field:', options=('field01_src', 'field02_src', 'field03_src', 'fi…

<function __main__.plot_field(field_name)>